# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [20]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

## Data for the chosen tickers

In [21]:
TRAIN_START_DATE = '2023-01-01'
TRAIN_END_DATE = '2024-07-01'
TRADE_START_DATE = '2024-07-01'
TRADE_END_DATE = '2025-06-08'

In [22]:

# 台股標的：台積電、聯發科、廣達、鴻海
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['2330.TW', '2454.TW', '2382.TW', '2317.TW']).fetch_data()

'''
# 台股標的：台積電
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['2330.TW']).fetch_data()
'''
'''
# 台股標的：聯發科
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['2454.TW']).fetch_data()
'''
'''
# 台股標的：廣達
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['2382.TW']).fetch_data()
'''
'''
# 台股標的：鴻海
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = ['2317.TW']).fetch_data()

'''

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (2324, 8)


"\n# 台股標的：鴻海\ndf_raw = YahooDownloader(start_date = TRAIN_START_DATE,\n                     end_date = TRADE_END_DATE,\n                     ticker_list = ['2317.TW']).fetch_data()\n\n"

In [23]:
df_raw.head()

Price,date,close,high,low,open,volume,tic,day
0,2023-01-03,92.169464,92.913519,91.146394,92.913519,46594672,2317.TW,1
1,2023-01-03,438.210876,438.694553,428.537347,431.439406,14885824,2330.TW,1
2,2023-01-03,65.128059,65.760375,64.676408,64.947402,8323151,2382.TW,1
3,2023-01-03,543.007385,543.850564,523.614264,525.300623,4451795,2454.TW,1
4,2023-01-04,91.239395,91.797433,91.146390,91.146390,41500261,2317.TW,2


# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [24]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (608, 8)
Successfully added vix


Successfully added turbulence index


In [25]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [26]:
processed_full.head()

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2023-01-03,2317.TW,92.169464,92.913519,91.146394,92.913519,46594672.0,1.0,0.000000,93.019746,90.389113,0.0,-66.666667,100.0,92.169464,92.169464,22.90,0.0
1,2023-01-03,2330.TW,438.210876,438.694553,428.537347,431.439406,14885824.0,1.0,0.000000,93.019746,90.389113,0.0,-66.666667,100.0,438.210876,438.210876,22.90,0.0
2,2023-01-03,2382.TW,65.128059,65.760375,64.676408,64.947402,8323151.0,1.0,0.000000,93.019746,90.389113,0.0,-66.666667,100.0,65.128059,65.128059,22.90,0.0
3,2023-01-03,2454.TW,543.007385,543.850564,523.614264,525.300623,4451795.0,1.0,0.000000,93.019746,90.389113,0.0,-66.666667,100.0,543.007385,543.007385,22.90,0.0
4,2023-01-04,2317.TW,91.239395,91.797433,91.146390,91.146390,41500261.0,2.0,-0.020867,93.019746,90.389113,0.0,-66.666667,100.0,91.704430,91.704430,22.01,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [27]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

1368
860


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [28]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')